In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from brisc.manuscript_analysis import distance_between_cells as dist_cells
from brisc.exploratory_analysis.plot_summary_for_all_bc import compute_flatmap_coors

from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

from brisc.manuscript_analysis import distance_between_cells as dist_cells
from brisc.manuscript_analysis import spatial_plots_rabies as spatial
import matplotlib.cm as cm
from brisc.manuscript_analysis import barcodes_in_cells as bc_cells

matplotlib.rcParams["pdf.fonttype"] = 42  # for pdfs

In [ ]:
barseq_path=Path("Y:/")
main_path=Path("Z:/")
cells_df = pd.read_pickle(
    barseq_path / "processed/becalia_rabies_barseq/BRAC8498.3e/analysis/cell_barcode_df.pkl"
)
cells_df = cells_df[cells_df["all_barcodes"].notna()].copy()

use_bases = 10
def shorten_barcodes(barcodes):
    return [barcode[:use_bases] for barcode in barcodes]

cells_df["all_barcodes"] = cells_df["all_barcodes"].apply(shorten_barcodes)
cells_df = bc_cells.find_singleton_bcs(cells_df)

relative_presyn_coords, distances = dist_cells.determine_presynaptic_distances(cells_df)

In [ ]:
multibarcoded_starters = bc_cells.analyze_multibarcoded_starters(cells_df)

In [ ]:
flat_coors = compute_flatmap_coors(cells_df)
cells_df["flatmap_x"] = flat_coors[:, 0]
cells_df["flatmap_y"] = flat_coors[:, 1]
cells_df["flatmap_z"] = flat_coors[:, 2]


In [ ]:
all_shuffled_distances, all_starter_coords = (
    dist_cells.create_barcode_shuffled_nulls_parallel(
        cells_df, N_iter=20
    )
)

In [ ]:
relative_presyn_coords_flatmap, distancess_flatmap = dist_cells.determine_presynaptic_distances(cells_df, col_prefix="flatmap_")
all_shuffled_distances_flatmap, _ = (
    dist_cells.create_barcode_shuffled_nulls_parallel(
        cells_df, N_iter=20, col_prefix="flatmap_"
    )
)

In [ ]:
atlas_size = 10
bin_image = spatial.prepare_area_labels(atlas_size=10)


In [ ]:
cortex_bin_image = spatial.prepare_area_labels(
    atlas_size=10,
    structures=[ 
        "root", "CTX", "MB", "DG", "DG-mo", "DG-sg", 
        "SCdg", "SCdw", "SCig", "SCiw", "SCop", "SCsg", 
        "SCzo", "PAG", "MRN", "TH", "RN", "VISp", "VISl",
        "VISli", "VISal", "VISam", "VISpm", "AUDp", "AUDd",
        "RSP", "TEa", "AUDpo", "AUDv", "ECT", "ENTl"
    ],
    xpos=861,
)


In [ ]:
# Plot Fig.1
fontsize_dict = {"title": 8, "label": 8, "tick": 6, "legend": 6}
pad_dict = {"label": 1, "tick": 1, "legend": 5}
line_width = 0.9
line_alpha = 1

cm = 1 / 2.54
fig = plt.figure(figsize=(17.4 * cm, 17.4 * cm))
data_path = main_path  / "home/shared/projects/barcode_diversity_analysis/collapsed_barcodes/"
save_path = main_path / "home/shared/presentations/becalick_2025"

save_fig = True
figname = "fig3_presynaptic_networks"




ax_coronal = fig.add_axes([0.02, 0.75, 0.3, 0.22])
ax_flatmap = fig.add_axes([0.29, 0.82, 0.25, 0.15])

spatial.plot_all_rv_cells(
    cells_df,
    ax_coronal,
    ax_flatmap,
    bin_image,
    legend_fontsize=6,
    atlas_size=10,
    area_colors={
        'AUDp': "limegreen", 
        'AUDpo': "mediumseagreen", 
        'AUDv': "springgreen", 
        'RSP': "darkorchid", 
        'TEa': "forestgreen", 
        'TH': "orangered", 
        'VISal': "aquamarine", 
        'VISl': "darkturquoise", 
        'VISli': "mediumaquamarine",
        'VISp': "deepskyblue", 
        'VISpm': "royalblue", 
        'fiber_tract': "gray",
        'non_cortical': "lightgray",
    }
)

ax_v1_rabies = fig.add_axes([0.60, 0.78, 0.1, 0.15])
ax_density = fig.add_axes([0.71, 0.78, 0.08, 0.15])
spatial.plot_rabies_cells(
    ax_v1_rabies,
    ax_density,
    cells_df,
    label_fontsize=fontsize_dict["label"],
    tick_fontsize=fontsize_dict["tick"],
)


ax_stack = fig.add_axes([0.9, 0.78, 0.10, 0.15])
bc_cells.plot_multibarcoded_starters(
    multibarcoded_starters,
    ax_stack,
    label_fontsize=fontsize_dict["label"],
    tick_fontsize=fontsize_dict["tick"],
    legend_fontsize=fontsize_dict["legend"],
    barcode_proportion=False,
)


barcodes =  ["TGTCACTTGC", "GCTTCATGCA", "TACAGCCAAA", "GCTCTTCCTT", "ATAAATAAGG" ]
ax_coronal_examples = fig.add_axes([0.02, 0.4, 0.45, 0.33])
ax_flatmap_examples = fig.add_axes([0.42, 0.43, 0.45, 0.33])

spatial.plot_example_barcodes(
    cells_df,
    ax_coronal_examples,
    ax_flatmap_examples,
    bin_image,
    barcodes=barcodes,
    barcode_colors=["crimson", "dodgerblue", "darkviolet", "orangered", "forestgreen"],
    legend_fontsize=6,
    atlas_size=10,
    starter_marker_size=25,
    presynaptic_marker_size=3,
)



# ax_dist_between_starters = fig.add_axes(ax_locations[8])
# im = dist_cells.plot_dist_between_starters(
#     distances_diff_bc_nonadj,
#     distances_same_bc_nonadj,
#     ax=ax_dist_between_starters,
#     bins=40,
#     max_dist=2,
#     label_fontsize=fontsize_dict["label"],
#     tick_fontsize=fontsize_dict["tick"],
#     line_width=line_width,
# )







# 1 Presynaptic distance shared barcode
# 2 Presynaptic distance shuffled barcode



if save_fig:
    fig.savefig(save_path / f"{figname}.pdf", format="pdf", transparent=True, bbox_inches="tight")
    fig.savefig(save_path / f"{figname}.png", format="png", transparent=True, bbox_inches="tight")

In [ ]:
# Coord format: [left, bottom, width, height]
ax_locations = [
    [0.05, 0.67, 0.25, 0.25],  # Row 1, Col 1
    [0.46, 0.67, 0.25, 0.25],  # Row 1, Col 2
    [0.05, 0.49, 0.25, 0.25],  # Row 2, Col 1
]
fig = plt.figure(figsize=(7, 7), dpi=300)

ax_presyn_shared = fig.add_axes([0.5, 0.5, 0.22, 0.22])

dist_cells.plot_relative_coors(
    relative_presyn_coords_flatmap / 100,
    ax=ax_presyn_shared,
    s=1,
    alpha=0.1,
    color="black",
    label_fontsize=fontsize_dict["label"],
    tick_fontsize=fontsize_dict["tick"],
    coors_to_plot=(0, 2),
    lims=((-5000, 5000), (-1000, 1000))
)
ax_presyn_shared.set_xlabel("")
# remove xtick labels
ax_presyn_shared.set_xticklabels([])
ax_presyn_shared.set_ylabel("")
ax_presyn_shuffled = fig.add_axes([0.5, 0.40, 0.22, 0.22])

dist_cells.plot_relative_coors(
    all_shuffled_distances_flatmap[0] / 100,
    ax=ax_presyn_shuffled,
    s=1,
    alpha=0.1,
    color="black",
    label_fontsize=fontsize_dict["label"],
    tick_fontsize=fontsize_dict["tick"],
    coors_to_plot=(0, 2),
    lims=((-5000, 5000), (-1000, 1000))
)
ax_presyn_shuffled.set_ylabel("                   Columnar location (um)")

# ax_dist_histo = fig.add_axes(ax_locations[1])
# dist_cells.plot_3d_distance_histo(
#     distances,
#     all_shuffled_distances,
#     ax=ax_dist_histo,
#     label_fontsize=fontsize_dict["label"],
#     tick_fontsize=fontsize_dict["tick"],
#     bins=100,
#     max_dist=5,
#     linewidth=0.9,
# )

In [ ]:
fig = plt.figure(figsize=(7, 7), dpi=300)

ax_presyn_shared = fig.add_axes([0.5, 0.5, 0.22, 0.22])

dist_cells.plot_relative_coors(
    relative_presyn_coords,
    ax=ax_presyn_shared,
    s=1,
    alpha=0.1,
    color="black",
    label_fontsize=fontsize_dict["label"],
    tick_fontsize=fontsize_dict["tick"],
    coors_to_plot=(2, 0)
)
ax_presyn_shared.set_xlabel("")
# remove xtick labels
ax_presyn_shared.set_xticklabels([])
ax_presyn_shared.set_ylabel("")
ax_presyn_shuffled = fig.add_axes([0.5, 0.40, 0.22, 0.22])

dist_cells.plot_relative_coors(
    all_shuffled_distances[0],
    ax=ax_presyn_shuffled,
    s=1,
    alpha=0.1,
    color="black",
    label_fontsize=fontsize_dict["label"],
    tick_fontsize=fontsize_dict["tick"],
    coors_to_plot=(2, 0)
)
ax_presyn_shuffled.set_ylabel("                   Columnar distance (um)")

In [ ]:
import seaborn as sns
plt.subplot(1,2,1)
# sns.kdeplot(relative_presyn_coords_flatmap[:, 2], bw_adjust=0.35)
# sns.kdeplot(relative_presyn_coords_flatmap[:, 1])
sns.kdeplot(relative_presyn_coords_flatmap[:, 1] / 100)

sns.kdeplot(all_shuffled_distances_flatmap[0][:, 1] / 100)
plt.xlim(-5, 5)
# sns.kdeplot(all_shuffled_distances[0][:, 1])
# sns.kdeplot(all_shuffled_distances_flatmap[0][:, 2], bw_adjust=0.35)
plt.subplot(1,2,2)
sns.kdeplot(relative_presyn_coords[:, 0])
sns.kdeplot(all_shuffled_distances[0][:, 0])
plt.xlim(-5, 5)

In [ ]:
import seaborn as sns
plt.subplot(1,2,1)
# sns.kdeplot(relative_presyn_coords_flatmap[:, 2], bw_adjust=0.35)
# sns.kdeplot(relative_presyn_coords_flatmap[:, 1])
sns.kdeplot(relative_presyn_coords_flatmap[:, 0] / 100)

sns.kdeplot(all_shuffled_distances_flatmap[0][:, 0] / 100)
plt.xlim(-5, 5)
# sns.kdeplot(all_shuffled_distances[0][:, 1])
# sns.kdeplot(all_shuffled_distances_flatmap[0][:, 2], bw_adjust=0.35)
plt.subplot(1,2,2)
sns.kdeplot(relative_presyn_coords_flatmap[:, 2] / 100)

sns.kdeplot(all_shuffled_distances_flatmap[0][:,2] / 100)

